<a href="https://colab.research.google.com/github/JakeEisner/ECON3916-Statistical-Machine-Learning/blob/main/Lab%206/%5BLab_6%5D_The_Architecture_of_Bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
# 1. Lab Objective: The Data Generating Process
"""
In previous labs, you treated data as "given." In Lab 5, we recognize that data is "taken." You will scrutinize the Data Generating Process (DGP) to understand how sampling mechanisms determine the validity of economic analysis.

The Economic Goal: To prove that simple random sampling creates "Sampling Error" in small datasets, and to engineer a fix using Stratification and Chi-Square Diagnostics.
"""

'\nIn previous labs, you treated data as "given." In Lab 5, we recognize that data is "taken." You will scrutinize the Data Generating Process (DGP) to understand how sampling mechanisms determine the validity of economic analysis.\n\nThe Economic Goal: To prove that simple random sampling creates "Sampling Error" in small datasets, and to engineer a fix using Stratification and Chi-Square Diagnostics.\n'

In [59]:
#2. Phase 1: The Danger of Randomness (Manual Split)
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Data Ingestion (The Population)
df = sns.load_dataset('titanic')
df
print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

# 2. Manual Shuffle (Simulation of Sampling)
# We set a seed to ensure reproducibility for the lesson,
# but in production, this variance happens naturally.
np.random.seed(2026)
indices = np.random.permutation(len(df))





Total Population: 891
Population Survival Rate: 0.3838


We will split the data 80/20. Then, we will calculate the survival rate in both groups. Ideally, they should be identical.

In [60]:
# Step 2: The Spilt and The Bias Check

# 3. Cut the deck (80/20 Split)
split_point = int(len(df) * 0.8)

# Slicing the shuffled indices
train_idx = indices[:split_point]
test_idx = indices[split_point:]

# Creating the subsets
train_set = df.loc[train_idx]
test_set = df.loc[test_idx]

# 4. Bias Check (The Delta)
train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate:  {test_surv:.4f}")
print(f"Sampling Bias (Delta): {delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate:  0.4246
Sampling Bias (Delta): 0.0510


3: Phase 2: Stratification (The Fix)

We suspect that "Class" (pclass) is a major driver of survival. A random split might accidentally put all First Class passengers in the Training set. We must force the distribution to be identical.

In [61]:
# Step 3: Fixing Covariate Shift
from sklearn.model_selection import train_test_split

# Stratify by 'pclass' ensures the distribution of classes is identical
X_train, X_test = train_test_split(df, test_size=0.2,stratify=df["pclass"], random_state = 42)

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.550562
1    0.242978
2    0.206461
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.553073
1    0.240223
2    0.206704
Name: proportion, dtype: float64


Step 4: The SRM Diagnostic (Forensics)
In "Tech Economics," data streams are often broken by engineering bugs. This leads to Sample Ratio Mismatch (SRM).

Action: Use the prompt below to generate a forensic tool that detects if an A/B test is biased.

In [62]:
import numpy as np
from scipy.stats import chisquare

# 1) Define observed and expected arrays
observed = np.array([450, 550])   # [Control, Treatment]
expected = np.array([500, 500])   # planned 50/50 out of 1000

# 2) Calculate Chi-Square statistic and p-value
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"Observed: {observed} | Expected: {expected}")
print(f"Chi-Square statistic: {chi2_stat:.4f}")
print(f"p-value: {p_value:.6f}")

# 3) Print conclusion
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.")
else:
    print("Variance is within natural limits.")


Observed: [450 550] | Expected: [500 500]
Chi-Square statistic: 10.0000
p-value: 0.001565
CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.
